In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [3]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

In [4]:
df_original.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'],
      dtype='object')

In [5]:
df_reducido = df_original.loc[:,['person','event','sku','model','city','region','country']]

In [6]:
df_region_person = df_original.loc[df_original['country'].notnull(),['person','region','country']]

In [7]:
df_region_person.head()

,person,region,country
2136629,4640420b,Parana,Brazil
2136630,4640420b,Parana,Brazil
2136631,4640420b,Unknown,Brazil
2136632,4640420b,Parana,Brazil
2136633,4640420b,Parana,Brazil


In [8]:
df_region_person.head()

,person,region,country
2136629,4640420b,Parana,Brazil
2136630,4640420b,Parana,Brazil
2136631,4640420b,Unknown,Brazil
2136632,4640420b,Parana,Brazil
2136633,4640420b,Parana,Brazil


In [9]:
df_region_person['counter'] = 1
df_prueba = df_region_person.groupby(['person','region'])['counter'].sum().reset_index()

In [10]:
df_region_person= df_prueba.sort_values(by='counter',ascending = False).drop_duplicates(subset='person',keep='first')

In [11]:
df_region_person = df_region_person.rename(columns={'region':'most_visited_region'})

In [12]:
df_region_person = df_region_person.drop(['counter'],axis=1)

In [13]:
df_reducido = df_reducido.merge(df_region_person, left_on='person', right_on='person', how='left')

In [14]:
df_reducido = df_reducido.loc[df_reducido['sku'].notnull(),['person','event','sku','model','most_visited_region']]

In [15]:
df_reducido.head()

,person,event,sku,model,most_visited_region
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,Rio de Janeiro
1,ad93850f,viewed product,304.0,iPhone 5s,Sao Paulo
2,0297fc1e,viewed product,6888.0,iPhone 6S,Rio de Janeiro
3,2d681dd8,viewed product,11890.0,iPhone 7,Sao Paulo
4,cccea85e,viewed product,7517.0,LG G4 H818P,Sao Paulo


In [16]:
df_prueba2 = df_reducido.groupby(['person','model']).size().reset_index()
df_prueba2.head()

,person,model,0
0,0008ed71,LG G4 H818P,1
1,0008ed71,Samsung Galaxy S6 Flat,1
2,0008ed71,iPhone SE,1
3,00091926,LG G4 H815P,1
4,00091926,LG G5 SE,1


In [17]:
df_prueba2 = df_reducido.merge(df_prueba2, how='left').rename(columns={0:'interacciones'})

In [18]:
df_prueba2.head()

,person,event,sku,model,most_visited_region,interacciones
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,Rio de Janeiro,5
1,ad93850f,viewed product,304.0,iPhone 5s,Sao Paulo,18
2,0297fc1e,viewed product,6888.0,iPhone 6S,Rio de Janeiro,114
3,2d681dd8,viewed product,11890.0,iPhone 7,Sao Paulo,6
4,cccea85e,viewed product,7517.0,LG G4 H818P,Sao Paulo,74


In [19]:
df_region_person = df_prueba2.sort_values(by='interacciones',ascending = False).drop_duplicates(subset='person',keep='first')

In [20]:
df_region_person.head()

,person,event,sku,model,most_visited_region,interacciones
518011,b1f4dbf6,viewed product,2694.0,iPhone 5s,Sao Paulo,836
513385,171e75cb,viewed product,6917.0,iPhone 6S,Espirito Santo,682
806048,6f19cfd9,viewed product,9021.0,iPhone 6S Plus,Bahia,624
406136,6abd2bf1,viewed product,1302.0,iPhone 6,Sao Paulo,607
1308678,9bf968c5,viewed product,1357.0,iPhone 6,Bahia,568


In [92]:
df_conversiones = df_original.loc[df_original['event'] == 'conversion', ['person','region', 'model', 'sku']]
df_conversiones.head()

,person,region,model,sku
195,49c19e32,NaN,iPhone 5c,2683.0
234,49c19e32,NaN,iPhone 5c,2683.0
724,39df97e0,NaN,Samsung Galaxy J2 Prime TV,9358.0
744,380c0e60,NaN,Samsung Galaxy Note 5,6314.0
1198,35ee0cc8,NaN,Samsung Galaxy A5 2017,10924.0


In [93]:
df_region_por_persona = df_original.loc[df_original['region'].notnull(),['person','region','country']]     

In [94]:
df_region_por_persona['contador'] = 1
df_region_por_persona = df_region_por_persona.groupby(['person','region'])['contador'].sum().reset_index()

In [95]:
df_region_por_persona = df_region_por_persona.sort_values(by='contador',ascending=False).drop_duplicates(subset='person',keep='first')

In [96]:
df_region_por_persona = df_region_por_persona.drop(columns='contador')

In [97]:
df_conversiones = df_conversiones.drop(columns='region')
df_conversiones = df_conversiones.merge(df_region_por_persona, on='person', how='left')
df_conversiones.head()

,person,model,sku,region
0,49c19e32,iPhone 5c,2683.0,Sao Paulo
1,49c19e32,iPhone 5c,2683.0,Sao Paulo
2,39df97e0,Samsung Galaxy J2 Prime TV,9358.0,Minas Gerais
3,380c0e60,Samsung Galaxy Note 5,6314.0,Sao Paulo
4,35ee0cc8,Samsung Galaxy A5 2017,10924.0,Unknown


In [203]:
df_ventas_por_region = df_conversiones.groupby(['region','sku']).size().reset_index().rename(columns={0:'Ventas'})

In [204]:
df_ventas_por_region.loc[df_ventas_por_region['region'] == 'Sao Paulo',:].head()

,region,sku,Ventas
2163,Sao Paulo,71.0,1
2164,Sao Paulo,74.0,2
2165,Sao Paulo,80.0,5
2166,Sao Paulo,87.0,5
2167,Sao Paulo,102.0,2


In [205]:
df_ventas_por_region = df_ventas_por_region.sort_values('Ventas', ascending=False).groupby('region').head(10)

In [206]:
df_ventas_por_region.loc[df_ventas_por_region['region'] == 'Sao Paulo',:]

,region,sku,Ventas
2404,Sao Paulo,6371.0,39
2276,Sao Paulo,2766.0,39
2401,Sao Paulo,6357.0,36
2403,Sao Paulo,6370.0,30
2187,Sao Paulo,290.0,24
2185,Sao Paulo,283.0,21
2345,Sao Paulo,3371.0,17
2593,Sao Paulo,9288.0,16
2205,Sao Paulo,506.0,15
2261,Sao Paulo,2699.0,14


In [208]:
df_agrupado = df_ventas_por_region.groupby('region')
top_10_regiones = {}
df_regiones_que_convirtieron = df_conversiones.drop_duplicates()['region'].dropna()
for region in df_regiones_que_convirtieron:
    top_10_regiones[region] = list(df_agrupado.get_group(region)['sku'])

In [209]:
top_10_regiones['Sao Paulo']

[6371.0, 2766.0, 6357.0, 6370.0, 290.0, 283.0, 3371.0, 9288.0, 506.0, 2699.0]

In [222]:
df_conversiones = df_conversiones.dropna(subset=['region'])

def asd(df, top_10_regiones):
    lista = []
    for s in df.itertuples():
        lista.append(s.sku in top_10_regiones[s.region])
    print(len(lista))
    return pd.Series(lista)

df_conversiones['compro_top_10'] = asd(df_conversiones,top_10_regiones)

7016


In [223]:
df_conversiones

,person,model,sku,region,compro_top_10
0,49c19e32,iPhone 5c,2683.0,Sao Paulo,False
1,49c19e32,iPhone 5c,2683.0,Sao Paulo,False
2,39df97e0,Samsung Galaxy J2 Prime TV,9358.0,Minas Gerais,False
3,380c0e60,Samsung Galaxy Note 5,6314.0,Sao Paulo,False
4,35ee0cc8,Samsung Galaxy A5 2017,10924.0,Unknown,True
5,c5a0bc36,Samsung Galaxy J7 2016 Metal,10855.0,Unknown,False
6,6f0826b2,iPhone 6S,6889.0,Rio de Janeiro,False
7,576d1af6,Samsung Galaxy S5,2741.0,Sao Paulo,False
8,04b13fd2,Motorola Moto G5 Plus,12550.0,Rio de Janeiro,False
9,bf15584a,iPhone 6S,6916.0,Sao Paulo,False


In [224]:
df_agrupado.get_group('Sao Paulo')

,region,sku,Ventas
2404,Sao Paulo,6371.0,39
2276,Sao Paulo,2766.0,39
2401,Sao Paulo,6357.0,36
2403,Sao Paulo,6370.0,30
2187,Sao Paulo,290.0,24
2185,Sao Paulo,283.0,21
2345,Sao Paulo,3371.0,17
2593,Sao Paulo,9288.0,16
2205,Sao Paulo,506.0,15
2261,Sao Paulo,2699.0,14
